In [1]:
%load_ext autoreload

import numpy as np
%aimport tafra
from tafra import Tafra

%autoreload 2

In [39]:
{'x': [1, 2, 3], 'x': [4, 5, 6]}

{'x': [4, 5, 6]}

In [37]:
np.append(np.empty_like(x['x'], shape=0))

TypeError: _append_dispatcher() missing 1 required positional argument: 'values'

In [2]:
t = Tafra({
    'x': np.array([1, 2, 3, 4, 5, 6]),
    'y': np.array(['one', 'two', 'one', 'two', 'one', 'two'], dtype='object'),
    'z': np.array([0, 0, 0, 1, 1, 1])
})
print('List:\n', t.to_list())
print('Record:\n', tuple(t.to_records()))


gb = t.group_by(
    ['y', 'z'], {'x': sum}, {'count': len}
)
print('\nGroup By:\n', gb)


tr = t.transform(
    ['y', 'z'], {'x': sum}, {'id': max}
)
print('\nTransform:\n', tr)


# tr = t.transform(
#     ['y', 'z'], {'x': sum, 'id': (max, '__id__')}
# )
# print('\nTransform with "magic" enumerator:\n', tr)

# can't do ^ that without magic __id__, but can do it as:
t2 = t.copy()
t2.id = np.empty_like(t2.x)
for i, (u, ix, grouped) in enumerate(t.iterate_by(['y', 'z'])):
    t2.x[ix] = sum(grouped.x)
    t2.id[ix] = i
print('\nTransformed without magic:\n', t2)

    
print('\nIterate By:\t')
for u, ix, grouped in t.iterate_by(['y']):
    print(grouped)


print('\nGroup By in Iterate By:\t')
for u, ix, grouped in t.iterate_by(['y']):
    print(grouped.group_by(['z'], {'x': sum}))

List:
 [array([1, 2, 3, 4, 5, 6]), array(['one', 'two', 'one', 'two', 'one', 'two'], dtype=object), array([0, 0, 0, 1, 1, 1])]
Record:
 ((1, 'one', 0), (2, 'two', 0), (3, 'one', 0), (4, 'two', 1), (5, 'one', 1), (6, 'two', 1))

Group By:
 Tafra(_data={'y': array(['one', 'two', 'two', 'one'], dtype=object), 'z': array([0, 0, 1, 1]), 'x': array([ 4,  2, 10,  5]), 'count': array([2, 1, 2, 1])}, _dtypes={'y': 'object', 'z': 'int', 'x': 'int', 'count': 'int'})

Transform:
 Tafra(_data={'y': array(['one', 'two', 'one', 'two', 'one', 'two'], dtype=object), 'z': array([0, 0, 0, 1, 1, 1]), 'x': array([ 4,  2,  4, 10,  5, 10]), 'id': array([0, 1, 0, 2, 3, 2])}, _dtypes={'y': 'object', 'z': 'int', 'x': 'int', 'id': 'int'})

Transformed without magic:
 Tafra(_data={'x': array([ 4,  2,  4, 10,  5, 10]), 'y': array(['one', 'two', 'one', 'two', 'one', 'two'], dtype=object), 'z': array([0, 0, 0, 1, 1, 1])}, _dtypes={'x': 'int', 'y': 'object', 'z': 'int'})

Iterate By:	
Tafra(_data={'x': array([1, 3, 5

In [3]:
_t = t.copy()
_t.update(_t.transform(['y'], {}, {'id': max}))

for u, ix, it in t.iterate_by(['y']):
    _t['x'][ix] = it['x'] - np.mean(it['x'])

print('\nIndex Use in Iterate By:')
print(_t)


Index Use in Iterate By:
Tafra(_data={'x': array([-2, -2,  0,  0,  2,  2]), 'y': array(['one', 'two', 'one', 'two', 'one', 'two'], dtype=object), 'z': array([0, 0, 0, 1, 1, 1]), 'id': array([0, 1, 0, 1, 0, 1])}, _dtypes={'x': 'int', 'y': 'object', 'z': 'int', 'id': 'int'})


In [4]:
print('\nGroup By in Iterate By:\t')
for u, ix, it in t.iterate_by(['y']):
    it['x'][0] = 9
    print(it)

print()
print(t)


Group By in Iterate By:	
Tafra(_data={'x': array([9, 3, 5]), 'y': array(['one', 'one', 'one'], dtype=object), 'z': array([0, 0, 1])}, _dtypes={'x': 'int', 'y': 'object', 'z': 'int'})
Tafra(_data={'x': array([9, 4, 6]), 'y': array(['two', 'two', 'two'], dtype=object), 'z': array([0, 1, 1])}, _dtypes={'x': 'int', 'y': 'object', 'z': 'int'})

Tafra(_data={'x': array([1, 2, 3, 4, 5, 6]), 'y': array(['one', 'two', 'one', 'two', 'one', 'two'], dtype=object), 'z': array([0, 0, 0, 1, 1, 1])}, _dtypes={'x': 'int', 'y': 'object', 'z': 'int'})


In [5]:
try:
    t.union(_t)
except Exception as e:
    print(e)

This `Tafra` column count does not match other `Tafra` column count.


In [6]:
t2 = t.copy()
t3 = t.copy()
t3.update_types({'x': float})

In [7]:
t.union(t2)

Tafra(_data={'x': array([1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6]), 'y': array(['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two', 'one',
       'two', 'one', 'two'], dtype=object), 'z': array([0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1])}, _dtypes={'x': 'int', 'y': 'object', 'z': 'int'})

In [8]:
try:
    t.union(t3)
except Exception as e:
    print(e)

This `Tafra` column `x` dtype `int` does not match other `Tafra` dtype `float`.


In [9]:
t3 = t.copy()
t2.union(t3, inplace=True)
t2

Tafra(_data={'x': array([1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6]), 'y': array(['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two', 'one',
       'two', 'one', 'two'], dtype=object), 'z': array([0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1])}, _dtypes={'x': 'int', 'y': 'object', 'z': 'int'})

In [10]:
x = t[:3]
x['x'][0] = 0
t['x']

array([0, 2, 3, 4, 5, 6])

In [11]:
x = t[slice(0, 3)]
x['x'][0] = 7
t['x']

array([7, 2, 3, 4, 5, 6])

In [12]:
z = t[:3].copy()
z['x'][0] = 9
t['x']

array([7, 2, 3, 4, 5, 6])

In [13]:
a = t[t['x'] <= 4]
a['x'][1] = 15
print(a['x'])
print(t['x'])

[ 2 15  4]
[7 2 3 4 5 6]


In [14]:
t.update_types({'x': float})

In [15]:
t.update_types({'x': 'float'})

In [16]:
try:
    t.update_types({'x': 'flot', 'y': 'st'})
except Exception as e:
    print(e)

`flot` is not a valid dtype for `x.`
`st` is not a valid dtype for `y.`



In [17]:
t.update_types({'x': int})
o = t.copy()
t.update(o)
t

Tafra(_data={'x': array([7, 2, 3, 4, 5, 6]), 'y': array(['one', 'two', 'one', 'two', 'one', 'two'], dtype=object), 'z': array([0, 0, 0, 1, 1, 1])}, _dtypes={'x': 'int', 'y': 'object', 'z': 'int'})

In [18]:
o.update_types({'x': float})
t.update(o)
t

Tafra(_data={'x': array([7., 2., 3., 4., 5., 6.]), 'y': array(['one', 'two', 'one', 'two', 'one', 'two'], dtype=object), 'z': array([0, 0, 0, 1, 1, 1])}, _dtypes={'x': 'float', 'y': 'object', 'z': 'int'})

In [19]:
o._data['x'] = np.arange(5)
o

Tafra(_data={'x': array([0, 1, 2, 3, 4]), 'y': array(['one', 'two', 'one', 'two', 'one', 'two'], dtype=object), 'z': array([0, 0, 0, 1, 1, 1])}, _dtypes={'x': 'float', 'y': 'object', 'z': 'int'})

In [20]:
try:
    t.update(o)
except Exception as e:
    print(e)
t

Other `Tafra` must have consistent row count. This `Tafra` has 6 rows, other `Tafra` has 5 rows.


Tafra(_data={'x': array([7., 2., 3., 4., 5., 6.]), 'y': array(['one', 'two', 'one', 'two', 'one', 'two'], dtype=object), 'z': array([0, 0, 0, 1, 1, 1])}, _dtypes={'x': 'float', 'y': 'object', 'z': 'int'})

In [21]:
try:
    t['x'] = list(range(6))
except Exception as e:
    print(e)
t

Tafra(_data={'x': array([0, 1, 2, 3, 4, 5]), 'y': array(['one', 'two', 'one', 'two', 'one', 'two'], dtype=object), 'z': array([0, 0, 0, 1, 1, 1])}, _dtypes={'x': 'int', 'y': 'object', 'z': 'int'})

In [22]:
t['x'] = np.arange(6)[:, None]

c:\users\dfulford\projects\tafra\tafra.py:169: UserWarning: `np.squeeze(ndarray)` applied to set ndim == 1.
  warnings.warn('`np.squeeze(ndarray)` applied to set ndim == 1.')


In [23]:
try:
    t['x'] = np.atleast_2d(np.arange(6))
except Exception as e:
    print(e)
t

c:\users\dfulford\projects\tafra\tafra.py:169: UserWarning: `np.squeeze(ndarray)` applied to set ndim == 1.
  warnings.warn('`np.squeeze(ndarray)` applied to set ndim == 1.')


Tafra(_data={'x': array([0, 1, 2, 3, 4, 5]), 'y': array(['one', 'two', 'one', 'two', 'one', 'two'], dtype=object), 'z': array([0, 0, 0, 1, 1, 1])}, _dtypes={'x': 'int', 'y': 'object', 'z': 'int'})

In [24]:
t['x'] = np.atleast_2d(np.arange(6)).T
t

c:\users\dfulford\projects\tafra\tafra.py:169: UserWarning: `np.squeeze(ndarray)` applied to set ndim == 1.
  warnings.warn('`np.squeeze(ndarray)` applied to set ndim == 1.')


Tafra(_data={'x': array([0, 1, 2, 3, 4, 5]), 'y': array(['one', 'two', 'one', 'two', 'one', 'two'], dtype=object), 'z': array([0, 0, 0, 1, 1, 1])}, _dtypes={'x': 'int', 'y': 'object', 'z': 'int'})

In [25]:
try:
    t['x'] = np.atleast_2d(np.arange(6))
except Exception as e:
    print(e)

c:\users\dfulford\projects\tafra\tafra.py:169: UserWarning: `np.squeeze(ndarray)` applied to set ndim == 1.
  warnings.warn('`np.squeeze(ndarray)` applied to set ndim == 1.')


In [26]:
try:
    t['x'] = np.repeat(np.arange(6)[:, None], repeats=2, axis=1)
except Exception as e:
    print(e)

`ndarray` or `np.squeeze(ndarray)` must have ndim == 1.


In [27]:
t['d'] = np.array([np.datetime64(_, 'D') for _ in range(6)])

In [28]:
t

Tafra(_data={'x': array([0, 1, 2, 3, 4, 5]), 'y': array(['one', 'two', 'one', 'two', 'one', 'two'], dtype=object), 'z': array([0, 0, 0, 1, 1, 1]), 'd': array(['1970-01-01', '1970-01-02', '1970-01-03', '1970-01-04',
       '1970-01-05', '1970-01-06'], dtype='datetime64[D]')}, _dtypes={'x': 'int', 'y': 'object', 'z': 'int', 'd': 'date'})

In [29]:
tuple(t.to_records())

((0, 'one', 0, '1970-01-01'),
 (1, 'two', 0, '1970-01-02'),
 (2, 'one', 0, '1970-01-03'),
 (3, 'two', 1, '1970-01-04'),
 (4, 'one', 1, '1970-01-05'),
 (5, 'two', 1, '1970-01-06'))

In [30]:
t.to_list()

[array([0, 1, 2, 3, 4, 5]),
 array(['one', 'two', 'one', 'two', 'one', 'two'], dtype=object),
 array([0, 0, 0, 1, 1, 1]),
 array(['1970-01-01', '1970-01-02', '1970-01-03', '1970-01-04',
        '1970-01-05', '1970-01-06'], dtype='datetime64[D]')]